In [219]:
import mysql.connector
import pandas as pd

In [221]:
# Connection to database
mydb = mysql.connector.connect(
  host = "127.0.0.1",
  user = "root",
  passwd = "",
  database ='newschema'
)
mycursor = mydb.cursor()

In [222]:
#Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do

mycursor.execute("SELECT name FROM Facilities WHERE membercost = 0")
                 
q1_result = mycursor.fetchall()
free_facilities = [] #Initializing empty list
for row in q1_result:
  free_facilities.append(row[0])
print("The facilities that are free for members are:")
print(*free_facilities, sep = '\n')

The facilities that are free for members are:
Badminton Court
Table Tennis
Snooker Table
Pool Table


In [223]:
#Q2: How many facilities do not charge a fee to members?
mycursor.execute("SELECT COUNT(*) FROM Facilities WHERE membercost = 0")
                 
q2_result = mycursor.fetchall()
free_facilities = [] #Initializing empty list
for row in q2_result:
  free_facilities.append(row[0])
print(*free_facilities, "facilities do not charge a fee for members", )

4 facilities do not charge a fee for members


In [224]:
# Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost?
# Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.  
mycursor.execute("SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost > 0 AND membercost < (monthlymaintenance * 0.2)")
q3_result = mycursor.fetchall()

facilities_list = []
for row in q3_result:
    facilities_list.append(row[1])
    
print("The facilities are:")
print(*facilities_list, sep='\n')

The facilities are:
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


In [227]:
# Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.
mycursor.execute("SELECT * FROM Facilities WHERE facid in (1,5)")
q4_result = mycursor.fetchall()

facilities_1_and_5 = []
for row in q4_result:
    facilities_1_and_5.append(row[1])

df_fac_1_and_5 = pd.DataFrame(q4_result, columns = ['ID', 'Name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance' ])
print(df_fac_1_and_5)

   ID            Name membercost guestcost  initialoutlay  monthlymaintenance
0   1  Tennis Court 2        5.0      25.0           8000                 200
1   5  Massage Room 2        9.9      80.0           4000                3000


In [228]:
# Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their 
# monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.
mycursor.execute("SELECT CASE WHEN monthlymaintenance >100 THEN  'expensive' ELSE  'cheap' END AS cost_category, name, monthlymaintenance FROM Facilities")
q5_result = mycursor.fetchall()

df_cheap_or_expensive = pd.DataFrame(q5_result, columns = ['Label', 'Name', 'Monthly Maintenance'])
print(df_cheap_or_expensive)


       Label             Name  Monthly Maintenance
0  expensive   Tennis Court 1                  200
1  expensive   Tennis Court 2                  200
2      cheap  Badminton Court                   50
3      cheap     Table Tennis                   10
4  expensive   Massage Room 1                 3000
5  expensive   Massage Room 2                 3000
6      cheap     Squash Court                   80
7      cheap    Snooker Table                   15
8      cheap       Pool Table                   15


In [229]:
#Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.
q6_cursor = mydb.cursor()
q6_cursor.execute("SELECT firstname, surname FROM Members WHERE joindate = (SELECT MAX( joindate ) FROM Members )")
q6_result = q6_cursor.fetchall()
print("First name:", q6_result[0][0])
print("Last name:", q6_result[0][1])

First name: Darren
Last name: Smith


In [231]:
# Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the 
# court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by 
# the member name.
q7_cursor = mydb.cursor()
q7_cursor.execute("SELECT DISTINCT concat(m.firstname, ' ', m.surname) as fullname FROM Facilities AS f LEFT JOIN Bookings as b ON f.facid = b.facid INNER JOIN Members as m ON m.memid = b.memid WHERE f.facid in (0, 1) ORDER BY fullname")
q7_result = q7_cursor.fetchall()

for name in q7_result:
    print(name[0])

Anne Baker
Burton Tracy
Charles Owen
Darren Smith
David Farrell
David Jones
David Pinker
Douglas Jones
Erica Crumpet
Florence Bader
Gerald Butters
GUEST GUEST
Henrietta Rumney
Jack Smith
Janice Joplette
Jemima Farrell
Joan Coplin
John Hunt
Matthew Genting
Millicent Purview
Nancy Dare
Ponder Stibbons
Ramnaresh Sarwin
Tim Boothe
Tim Rownam
Timothy Baker
Tracy Smith


In [233]:
# Q8: How can you produce a list of bookings on the day of 2012-09-14 which
# will cost the member (or guest) more than $30? Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.
q8_cursor = mydb.cursor()
q8_cursor.execute("SELECT f.name, concat(m.firstname, ' ', m.surname) as fulllname,(CASE WHEN b.memid >0 THEN b.slots * f.membercost ELSE b.slots * f.guestcost END) AS guest_or_member_cost FROM Facilities as f LEFT JOIN Bookings as b on b.facid = f.facid INNER JOIN Members as m on m.memid = b.memid WHERE b.starttime >= '2012-09-14 00:00:00' AND b.starttime <= '2012-09-14 23:59:59' HAVING guest_or_member_cost >30 ORDER BY guest_or_member_cost DESC")
q8_result = q8_cursor.fetchall()
q8_df = pd.DataFrame(q8_result, columns = ['Facility', 'Name', 'Cost'])
print(q8_df)

          Facility            Name   Cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 2     GUEST GUEST   75.0
7   Tennis Court 1     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Jemima Farrell   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


In [234]:
# Q9: This time, produce the same result as in Q8, but using a subquery.
q9_cursor = mydb.cursor()
q9_cursor.execute("SELECT sub1.name, concat(m.firstname, ' ', m.surname) as fulllname, (CASE WHEN b.memid >0 THEN b.slots * sub1.membercost ELSE b.slots * sub1.guestcost END) AS guest_or_member_cost FROM (SELECT name, facid, membercost, guestcost FROM Facilities) AS sub1 LEFT JOIN Bookings AS b on b.facid = sub1.facid INNER JOIN Members as m on m.memid = b.memid WHERE b.starttime >= '2012-09-14 00:00:00' AND b.starttime <= '2012-09-14 23:59:59' HAVING guest_or_member_cost >30 ORDER BY guest_or_member_cost DESC")
q9_result = q9_cursor.fetchall()
q9_df = pd.DataFrame(q9_result, columns = ['Facility', 'Name', 'Cost'])
print(q9_df)

          Facility            Name   Cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 2     GUEST GUEST   75.0
7   Tennis Court 1     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Jemima Farrell   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


In [235]:
# Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, 
# sorted by revenue. Remember that there's a different cost for guests and members!
q10_cursor = mydb.cursor()
q10_cursor.execute("SELECT f.name, SUM(CASE WHEN b.memid >0 THEN b.slots * f.membercost ELSE b.slots * f.guestcost END) AS revenue FROM Facilities as f LEFT JOIN Bookings as b on b.facid = f.facid INNER JOIN Members as m on m.memid = b.memid GROUP BY f.name HAVING revenue < 1000 ORDER BY revenue ")
q10_result = q10_cursor.fetchall()
q10_df = pd.DataFrame(q10_result, columns = ['Facility', 'Revenue'])
print(q10_df)


        Facility Revenue
0   Table Tennis   180.0
1  Snooker Table   240.0
2     Pool Table   270.0
